# Gene set and pathway

![](./images/Module3/Workflow3.jpg)

Differential expression (DE) analysis typically yields a list of genes or proteins that require further interpretation.
Our intention is to use such lists to gain novel insights about genes and proteins that may have roles in a given
phenomenon, phenotype or disease progression. However, in many cases, gene lists generated from DE analysis are
difficult to interpret due to their large size and lack of useful annotations. Hence, pathway analysis (also known
as gene set analysis or over-representation analysis), aims to reduce the complexity of interpreting gene lists
via mapping the listed genes to known (i.e. annotated) biological pathways, processes and functions.
This learning module introduces common curated biological databases including Gene Ontology (GO),
Kyoto Encyclopedia of Genes and Genomes (KEGG).

## Learning Objectives:
1. Introduction to Ontology and Gene Ontology.
2. Introduction to KEGG Pathway Database.
3. Downloading terms, pathway gene set from GO and KEGG.
4. Saving results to GMT file format.

## Ontology and Gene Ontology
### Overview
In this section we will learn about the concept of gene ontology in Bioinformatics. Ontology is a specification
of a conceptualization which is a set of concepts within a domain, defined by a shared vocabulary to
denote the types and properties of the concepts as well as the relationships between the concepts.
Ontology plays an important role in the field of bioinformatics. Ontology enable communicate unambiguously e.g.,
to understand across different groups’ annotations of various genomes. Also, it allows the knowledge to be represented
in a computable and structured to perform automated analyses by computer programs.

The Gene Ontology (GO) defines a structured, common, controlled vocabulary to describe attributes of genes and gene products
across organisms. Collaboration is key to build a consensus vocabulary. But the term gene ontology, or GO, is commonly used
to refer to both, which is sometimes a source of potential confusion. In order to avoid this, here we will use the term “GO”
to describe the set of terms and their hierarchical structure and “GO annotations” to describe the set of associations between
genes and GO terms. The GO is divided into three categories to describe the genes and gene products from three different
angles: Molecular Function, Biological Process, and Cellular Component.

The structure of GO can be described in terms of into directed acyclic graphs (DAGs), where each GO term is a node,
and the relationships between the terms are edges between the nodes. GO is loosely hierarchical, with ‘child’ terms
being more specialized than their ‘parent’ terms, but unlike a strict hierarchy, a term may have more than one parent
term (note that the parent/child model does not hold true for all types of relations). The structure of the controlled
vocabularies are intended to reflect true, biological relationships. In contrast to strict hierarchies, DAGS allow
multiple relationships between a more granular (child) term and a more general parent term. The relationship between
terms affects how queries are made. For example,a query for all genes with binding activity would include transcription
factors as well as genes with other types of binding activity (such as protein binding, ligand binding). The illustration
of category and structure of GO is shown in the figure below:

![](./images/Module3/GO_Structure.jpg)
*(Source: https://www.ebi.ac.uk/, http://geneontology.org/)*

### Gene ontology relationship
In DAGs graph, *terms* are represented as *nodes* and *relations* (also known as *object properties*) between the *terms*
are *edges*. There are commonly used relationships in GO such as *is a* (is a *subtype of*), *part of, has part, regulates,
negatively regulates and positively regulates*. All terms (except from the root terms representing each aspect) have an
is a sub-class relationship to another terms.

Examples:

```{admonition} *is a* relation
**GO:1904659:glucose transport** *is* a **GO:0015749:monosaccharide transport**.

The *is a* relation forms the basic structure of GO. If we say A *is a* B, we mean that node A is a subtype of node B

```

```{admonition} *is part of* relation
**GO:0031966:mitochondrial membrane** *is part of* **GO:0005740:mitochondrial envelope**

The *part of* relation is used to represent part-whole relationships. A *part of* relation would only be added between
A and B if B is **necessarily** *part of* A: wherever B exists, it is as *part of* A, and the presence of the B implies
the presence of A. However, given the occurrence of A, we cannot say for certain that B exists.
```

```{admonition} *regulates* relation
**GO:0098689:latency-replication decision** *regulates* **GO:0019046:release from viral latency**

A relation that describes case in which one process directly affects the manifestation of another process or quality,
i.e. the former *regulates* the latter.
```

A more specific case with more nodes and edges can be seen at the figure below:
<br>
![](./images/Module3/GO_Relation.jpg)
*(Source: https://advaitabio.com/)* <br>
For more technical information about relations and their properties used in GO and other ontologies see the
<a href="https://obofoundry.org/ontology/ro.html">OBO Relations Ontology (RO)</a>


### GO storage file formats
GO terms are updated monthly in the following formats:
* OBO 1.4 files are human-readable (in addition to machine-readable) and can be opened in any text editor.
* OWL files can be read by <a href="https://protege.stanford.edu/">Protégé</a> text editor.

 In this learning module, we will only use ".OBO" to obtain GO terms.The OBO file format is for representing ontologies and controlled vocabularies. The format itself attempts to achieve the following goals:
 * Human readability
 * Ease of parsing
 * Extensibility
 * Minimal redundancy

The file structure can is shown in the following figure.


![](./images/Module3/OBO_Format.jpg)

The OBO file has a header which is an unlabeled section at the beginning of the document. The header ends when the first term is encountered. Next, term is represented in labeled section with the tag *[Term]*. Under each term, we can find other information such as term ID, official name, category (namcespace), term definition, synonym and relation to other GO term.

At this step, we still don't know what genes are related to which GO terms. In order to retrieve custom sets of gene ontology annotations for any list of genes from organisms, NCBI has published a Gene2GO database that obtain GO terms and the entrez gene ids related to those go terms. The database can be retrieve from <a href="https://ftp.ncbi.nih.gov/gene/DATA/gene2go.gz">here</a> text editor. The Gene2GO database can be viewed using text editor, the file structure is presented in the figure below:

![](./images/Module3/Gene2GO.jpg)

The OBO and Gene2GO databases will be used in combination to obtain GO term and related genes for enrichment analysis.

### Retrieving GO terms from DE gene list
This section focuses on downloading related GO terms based on the DE genelist obtained from the DE analysis in the previous section.
Here, we will use `toGO` and `hgu133plus2.db` R packages to obtain GO terms. The `topGO` package has built-in functions that use Gene2GO databases to retrieve GO terms from the gene ID give by DE analysis. Since the dataset we used in the Module 02 was generated for human, we will use `hgu133plus2.db` database to map probeID to gene symbols.
The installation process of two package can be done by the script below:

In [1]:
# Installation of topGO and hgu133plus2.db package
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("topGO", update = F))
  suppressWarnings(BiocManager::install("hgu133plus2.db", update = F))
  suppressWarnings(BiocManager::install("AnnotationDbi", update = F))
})

In [2]:
# Loading the library
suppressPackageStartupMessages({
  suppressWarnings(library("topGO"))
  suppressWarnings(library("hgu133plus2.db"))
  suppressWarnings(library("AnnotationDbi"))
})


groupGOTerms: 	GOBPTerm, GOMFTerm, GOCCTerm environments built.



Load the DE genelist generated from the DE analysis using `limma`.

In [3]:
# Loading the DE result
data = readRDS("./data/DE_genes.rds")

By default, the DE analysis performed by `limma` contains multiple features. However, adjusted *p-value* and gene ID are the most important features for enrichment analysis. We can use the following code to list of gene IDs and their *p-value*.

In [4]:
# Get p-value from DE results
genelist <- data$adj.P.Val
# Assgin gene IDs to associated p-values
names(genelist) <- data$PROBEID

After successfully obtaining the genelist, we need to map the gene IDs to the gene symbols using `hgu133plus2.db`.

In [5]:
# Map gene IDs to gene symbols
gene <- suppressMessages(AnnotationDbi::select(hgu133plus2.db, names(genelist), "SYMBOL"))

In [6]:
# Remove duplicated gene IDs
gene <- gene[!duplicated(gene[,1]),]

# Assign result to a new genlist with gene symbols
geneList2 <- genelist
names(geneList2) <- gene[,2]

Now, we can search for related GO terms based on the new gene list using `topGO` package. First, we need to create a `topGOdata`
object.

In [7]:
# Retrieve all the GO terms related to the genelist obtained from the expression matrix
GOdata <- new("topGOdata", description = "",ontology = "BP",
                    allGenes = geneList2, geneSel = function(x)x, nodeSize = 10,
                    annot = annFUN.org, ID = "alias", mapping = "org.Hs.eg")


Building most specific GOs .....

	( 12301 GO terms found. )


Build GO DAG topology ..........

	( 15901 GO terms and 36216 relations. )


Annotating nodes ...............

	( 16201 genes annotated to the GO terms. )



We can search for related GO terms using `geneInTerm` function and view the term with associated genes.

In [8]:
# Obtain a list of genes for each GO term
allGO = genesInTerm(GOdata)
allGO[1:5]

$`GO:0000002`
 [1] "AKT3"     "DNA2"     "LIG3"     "LONP1"    "MEF2A"    "MGME1"   
 [7] "MPV17"    "OPA1"     "PARP1"    "PIF1"     "PIM1"     "POLB"    
[13] "POLG2"    "PPARGC1A" "PRIMPOL"  "SESN2"    "SLC25A33" "SLC25A36"
[19] "SLC25A4"  "STOX1"    "TOP3A"    "TP53"     "TYMP"    

$`GO:0000003`
   [1] "A1CF"      "AAAS"      "ABAT"      "ABCC2"     "ABCC8"     "ABHD2"    
   [7] "ACE"       "ACOD1"     "ACOX1"     "ACR"       "ACRBP"     "ACSBG2"   
  [13] "ACSL4"     "ACTR1A"    "ACTR2"     "ACTR3"     "ACVR1"     "ACVR1B"   
  [19] "ACVR1C"    "ACVR2A"    "ADA"       "ADAD1"     "ADAM10"    "ADAM15"   
  [25] "ADAM18"    "ADAM19"    "ADAM2"     "ADAM20"    "ADAM21"    "ADAM28"   
  [31] "ADAM29"    "ADAM32"    "ADAM5"     "ADAMTS1"   "ADAMTS16"  "ADAMTS2"  
  [37] "ADCY10"    "ADCY3"     "ADCY7"     "ADCYAP1"   "ADCYAP1R1" "ADGRG1"   
  [43] "ADGRG2"    "ADIPOR2"   "ADM"       "ADNP"      "ADRA2B"    "ADRA2C"   
  [49] "AFF4"      "AFP"       "AGFG1"     "AGO2"      "AGO4"      "AGRP"     
  [55] "AGT"       "AKAP3"     "AKAP4"     "AKR1B1"    "AKR1C3"    "AKT1"     
  [61] "ALDOA"     "ALKBH1"    "ALKBH5"    "ALOX15B"   "ALPL"      "AMBP"     
  [67] "AMH"       "AMHR2"     "ANG"       "ANGPT2"    "ANKRD31"   "ANKRD49"  
  [73] "ANTXR1"    "ANXA1"     "AP3B1"     "APELA"     "APOB"      "APOL2"    
  [79] "APOL3"     "APP"       "AR"        "AREG"      "ARG1"      "ARHGDIB"  
  [85] "ARID4A"    "ARID5B"    "ARMC2"     "ARNT"      "ARNTL"     "ARRB1"    
  [91] "ARRB2"     "ASB1"      "ASCL2"     "ASF1B"     "ASH1L"     "ASPM"     
  [97] "ASZ1"      "ATAT1"     "ATM"       "ATP1A1"    "ATP1A4"    "ATP2B2"   
 [103] "ATP2B4"    "ATP7A"     "ATP8B3"    "ATR"       "ATRX"      "AURKA"    
 [109] "AURKC"     "AVP"       "AVPR1A"    "AXL"       "AZI2"      "AZIN2"    
 [115] "B4GALNT1"  "B4GALT1"   "BACH1"     "BAD"       "BAG6"      "BAK1"     
 [121] "BAP1"      "BASP1"     "BAX"       "BBS2"      "BBS4"      "BCAP31"   
 [127] "BCKDK"     "BCL2"      "BCL2L1"    "BCL2L10"   "BCL2L11"   "BCL2L2"   
 [133] "BCL6"      "BIK"       "BIRC2"     "BIRC3"     "BIRC6"     "BMP15"    
 [139] "BMP4"      "BMP5"      "BMP6"      "BMP7"      "BMPR1A"    "BMPR1B"   
 [145] "BMPR2"     "BNC1"      "BOK"       "BOLL"      "BPTF"      "BPY2"     
 [151] "BRCA2"     "BRD2"      "BRDT"      "BRF1"      "BRINP1"    "BRIP1"    
 [157] "BRME1"     "BSG"       "BTBD18"    "BTG1"      "BUB1"      "BUB1B"    
 [163] "BUB3"      "C11orf80"  "C14orf39"  "C16orf92"  "C1QBP"     "C3"       
 [169] "C9orf24"   "CABS1"     "CABYR"     "CACNA1H"   "CACNA1I"   "CAD"      
 [175] "CADM1"     "CALCA"     "CALR"      "CALR3"     "CAPN2"     "CASP2"    
 [181] "CASP3"     "CASP8"     "CATSPER1"  "CATSPER2"  "CATSPER3"  "CATSPERB" 
 [187] "CATSPERD"  "CATSPERG"  "CBL"       "CBX2"      "CCDC134"   "CCDC136"  
 [193] "CCDC182"   "CCDC39"    "CCDC40"    "CCDC42"    "CCDC63"    "CCDC87"   
 [199] "CCIN"      "CCN1"      "CCNA1"     "CCNB1"     "CCNB1IP1"  "CCNB2"    
 [205] "CCNB3"     "CCND1"     "CCNE1"     "CCNE2"     "CCNF"      "CCNI"     
 [211] "CCR6"      "CCT2"      "CCT3"      "CCT4"      "CCT5"      "CCT7"     
 [217] "CCT8"      "CD38"      "CD46"      "CD9"       "CDC20"     "CDC25A"   
 [223] "CDC25B"    "CDC25C"    "CDH1"      "CDK1"      "CDK16"     "CDK2"     
 [229] "CDKL2"     "CDKN1B"    "CDX2"      "CDX4"      "CDY1"      "CDYL"     
 [235] "CEBPA"     "CEBPB"     "CECR2"     "CELF1"     "CELF3"     "CELF4"    
 [241] "CENPC"     "CENPI"     "CENPS"     "CENPX"     "CEP131"    "CEP57"    
 [247] "CETN2"     "CFAP157"   "CFAP206"   "CFAP251"   "CFAP43"    "CFAP44"   
 [253] "CFAP45"    "CFAP52"    "CFAP54"    "CFAP58"    "CFAP65"    "CFAP69"   
 [259] "CFAP97D1"  "CFTR"      "CGA"       "CGB3"      "CHD5"      "CHD7"     
 [265] "CHFR"      "CIB1"      "CITED1"    "CITED2"    "CKS2"      "CLDN11"   
 [271] "CLDN4"     "CLGN"      "CLIC4"     "CLIC5"     "CLOCK"     "CNBD2"    
 [277] "CNOT9"     "CNP"       "CNR1"      "CNTD1"     "CNTFR"  

Now, we already had GO terms with genes. However, we still do not know the meaning of GO terms related to biological process. We can use `GO.db` database to get a set of annotation maps describing the entire Gene Ontology assembled using data from GO. We can use the following code to install the `GO.db` R package.

In [9]:
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("GO.db", update = F))
})
library(GO.db)

Then, we can use the following command to obtain the GO terms description.

In [10]:
# Getting the name of each GO term
terms <- names(allGO)
# Getting the description of each GO term
descriptions <-lapply(Term(terms), `[[`, 1)

In order to perform enrichment analysis in later module, we need to save the GO terms and genesets to the standard output. One commonly used format is Gene Matrix Transposed file format *(\*.gmt)*. The GMT file format is a tab delimited file format that describes gene sets. In the GMT format, each row represents a gene set; in the GMX format, each column represents a gene set. Here, we can save GO terms and genesets to the *\*.gmt* using the following function:

In [11]:
# A function to save the GO terms with geneset to the local repository
writeGMT <- function(genesets, descriptions, outfile) {

  if (file.exists(outfile)) {
    file.remove(outfile)
  }
  for (gs in names(genesets)) {
    write(c(gs, gsub("\t", " ", descriptions[[gs]]), genesets[[gs]]), file=outfile, sep="\t", append=TRUE, ncolumns=length(genesets[[gs]]) + 2)
  }
}
outfile <- "./data/GO_terms.gmt"
writeGMT(allGO, descriptions, outfile)

Terminal command to save to Google Cloud Bucket
```{admonition} Saving to Google Cloud Bucket
gsutil cp ./data/GO_terms.gmt gs://cpa-output
```

## Kyoto Encyclopedia of Genes and Genomes (KEGG)
### Overview
KEGG is a collection of databases dealing with genomes, biological pathways, diseases, drugs, and chemical substances. KEGG is utilized for bioinformatics research and education, including data analysis in genomics, metagenomics, metabolomics and other omics studies, modeling and simulation in systems biology, and translational research in drug development. The KEGG database project was initiated in 1995 by Minoru Kanehisa, professor at the Institute for Chemical Research, Kyoto University, under the then ongoing Japanese Human Genome Program. Foreseeing the need for a computerized resource that can be used for biological interpretation of genome sequence data, he started developing the KEGG PATHWAY database. It is a collection of manually drawn KEGG pathway maps representing experimental knowledge on metabolism and various other functions of the cell and the organism. Each pathway map contains a network of molecular interactions and reactions and is designed to link genes in the genome to gene products (mostly proteins) in the pathway. This has enabled the analysis called KEGG pathway mapping, whereby the gene content in the genome is compared with the KEGG PATHWAY database to examine which pathways and associated functions are likely to be encoded in the genome. KEGG is a "computer representation" of the biological system. It integrates building blocks and wiring diagrams of the system—more specifically, genetic building blocks of genes and proteins, chemical building blocks of small molecules and reactions, and wiring diagrams of molecular interaction and reaction networks. The illustrative structure of KEGG is presented as figure below.
![](./images/Module3/KEGG.jpg)


### Retrieving pathways from KEGG databases
In this section, we will retrieve pathways and related gensets from the KEGG database using R command line. Here we will use `KEGGREST` R package that provides a client interface to the KEGG REST server. `KEGGREST` can be installed from the Bioconductor using following command.

In [12]:
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("KEGGREST", update = F))
})
suppressPackageStartupMessages({
  library(KEGGREST)
})

KEGG exposes a number of databases. To get an idea of what is available, run `listDatabases()`:

In [13]:
KEGGREST::listDatabases()

[1] "pathway"  "brite"    "module"   "ko"       "genome"   "vg"      
 [7] "ag"       "compound" "glycan"   "reaction" "rclass"   "enzyme"  
[13] "disease"  "drug"     "dgroup"   "environ"  "genes"    "ligand"  
[19] "kegg"

We can use these databases in further queries. Note that in many cases you can also use a three-letter KEGG organism code or a “T number” (genome identifier) in the same place you would use one of these database names.

We can obtain the list of organisms available in KEGG with the `keggList()` function:

In [14]:
organism <- keggList("organism")

In [15]:
print(paste0("KEGG supports ",dim(organism)[1]," organisms"))

[1] "KEGG supports 8524 organisms"


To view the supported organism we can use the following command:

In [16]:
# View several supported organism
head(organism)

T.number,organism,species,phylogeny
T01001,hsa,Homo sapiens (human),Eukaryotes;Animals;Vertebrates;Mammals
T01005,ptr,Pan troglodytes (chimpanzee),Eukaryotes;Animals;Vertebrates;Mammals
T02283,pps,Pan paniscus (bonobo),Eukaryotes;Animals;Vertebrates;Mammals
T02442,ggo,Gorilla gorilla gorilla (western lowland gorilla),Eukaryotes;Animals;Vertebrates;Mammals
T01416,pon,Pongo abelii (Sumatran orangutan),Eukaryotes;Animals;Vertebrates;Mammals
T03265,nle,Nomascus leucogenys (northern white-cheeked gibbon),Eukaryotes;Animals;Vertebrates;Mammals


In the Module 02, we performed DE analysis on human dataset. Therefore, we need to download pathways for human, the abbreviation of human pathway in KEGG is `hsa` and we can use `keggList` function to get the pathway list.

In [17]:
# Obtain the pathways belong to human
pathways.list <- keggList("pathway", "hsa")

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): Failure when receiving data from the peer


The pathway list contains pathway description and pathway code in a single line of text. To see the first five pathways, we can use the following command:

In [ ]:
# View the first five pathways
pathways.list[1:5]

We can see that, in each line, the text in the quotation mark contain pathway information while the later part cotains pathway code leading by a prefix `path:`. To get  pathway codes from the pathway list, we can use the following command:

In [ ]:
# Retrieve all the pathway IDs belong to human
pathway.codes <- sub("path:", "", names(pathways.list))
pathway.codes

We can use the following command to check how many pathways are available for human

In [ ]:
print(paste0("Number of available pathways for human are: ", length(pathway.codes)))

The following code will help to get list of genes and pathway's description for all pathways available in human.

In [ ]:
# Function to get all the gene names for each pathway
genes.by.pathway <- sapply(pathway.codes,
                           function(pwid){
                             pw <- keggGet(pwid)
                             if (is.null(pw[[1]]$GENE)) return(NA)
                             pw2 <- pw[[1]]$GENE[c(FALSE,TRUE)]
                             pw2 <- unlist(lapply(strsplit(pw2, split = ";", fixed = T), function(x)x[1]))
                             return(pw2)
                           }
)
# Function to get description for each pathway
description.by.pathway <- sapply(pathway.codes,
                                 function(pwid){
                                   pw <- keggGet(pwid)
                                   if (is.null(pw[[1]]$NAME)) return(NA)
                                   pw2 <- pw[[1]]$NAME
                                   return(pw2)
                                 }
)
# Convert the pathway description to a list
description.by.pathway <- as.list(description.by.pathway)

We can view the first five pathways with their genesets using the following command

Then we can save the output to *\*.gmt* file using the following commands

In [ ]:
# View the first five pathway with the genesets
genes.by.pathway[1:5]

Use the following command to see the description of the first five pathways

In [ ]:
# View the description of the first five pathways
description.by.pathway[1:5]

In [ ]:
# Saving the pathway information to the local repository
outfile <- "./data/KEGG_pathways.gmt"
writeGMT(genes.by.pathway, description.by.pathway, outfile)


Terminal command to save to Google Cloud Bucket
```{admonition} Saving to Google Cloud Bucket
gsutil cp ./data/KEGG_pathways.gmt gs://cpa-output
```